In [1]:
from numpy import ndarray

from stockfish import Stockfish

In [ ]:
# settings
settings = {
    "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 8, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 4096, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 0,
    "Minimum Thinking Time": 100,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 3500
}

In [ ]:
stockfish = Stockfish(depth=20, parameters=settings)
stockfish

In [ ]:
stockfish.set_fen_position("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")

In [ ]:
stockfish.get_top_moves(5)

In [ ]:
stockfish.get_top_moves(3)

In [ ]:
print(stockfish.get_board_visual())

In [ ]:
stockfish.make_moves_from_current_position(["d2d4"])

In [ ]:
print(stockfish.get_board_visual())

In [ ]:
stockfish.get_top_moves(10)

In [ ]:
stockfish.set_elo_rating(3500)
stockfish.get_top_moves(10)

In [ ]:
import platform
import numpy as np
platform.system()

In [ ]:
stockfish.get_evaluation()

In [ ]:
stockfish.set_fen_position("6qk/8/8/8/8/8/Q6p/7K b - - 0 1")

In [ ]:
stockfish.get_top_moves(10)

In [ ]:
import json
import numpy as np
import pandas as pd
from weighted_directed_graph import weighted_directed_graph
with open("output/test/1") as file:
    s = file.read().replace("'", "\"")
    dc = json.loads(s)
    print(dc)

In [ ]:
from typing import Any

states = [(-float("inf"), -20), (-20, -10), (-10, -5), (-5, -3), (-3, -1), (-1, -0.5), (-0.5, 0.5), (0.5, 1), (1, 3), (3, 5), (5, 10), (10, 20), (20, float("inf"))]
states_as_string = list(map(lambda x: f"{x}", states))
weights = None

with open("output/test2/0.json") as file:
    s = file.read().replace("'", "\"")
    weights = json.loads(s)

print(weights)
all_states = ["-1"] + states_as_string + ["1"]

def build_adjacency_matrix(weights):
    num_states = len(states_as_string) + 2
    all_states = ["-1"] + states_as_string + ["1"]
    state_to_index_mapping = dict()
    matrix = np.zeros(shape=(num_states, num_states))

    for i, state_name in enumerate(all_states):
        state_to_index_mapping[state_name] = i

    for edge, weight in weights.items():
        u, v = edge.split("|")
        u_ind = state_to_index_mapping[u]
        v_ind = state_to_index_mapping[v]
        matrix[u_ind, v_ind] = weight

    return matrix

def build_stochastic_matrix(adjacency_matrix: np.ndarray) -> Exception | Any:
    n, d = adjacency_matrix.shape
    if n != d:
        return Exception(f"n: {n} is not equal to d: {d} in adjacency matrix.")
    stochastic_matrix = np.zeros(shape=(n, n))
    for i in range(n):
        total_in_row = 0
        for j in range(n):
            total_in_row += adjacency_matrix[i, j]
        if total_in_row != 0:
            for j in range(n):
                stochastic_matrix[i, j] = adjacency_matrix[i, j] / total_in_row
    return stochastic_matrix

adjacency_matrix = build_adjacency_matrix(weights)
stochastic_matrix = build_stochastic_matrix(adjacency_matrix)
stochastic_matrix

In [ ]:
pd.DataFrame(stochastic_matrix, columns=all_states, index=all_states)

In [ ]:
inputs = [f"output/test2/{i}.json" for i in range(5)]
n = 15
m = np.zeros(shape=(5, 15))

for i in range(len(inputs)):
    with open(inputs[i]) as file:
        s = file.read().replace("'", "\"")
        weights = json.loads(s)
        adj = build_adjacency_matrix(weights)
        stoch = build_stochastic_matrix(adj)
        m[i] = stoch[stoch.shape[0] // 2]

pd.DataFrame(m, columns=all_states)

In [ ]:
stockfish.set_fen_position("r1bqkb1r/p2n1p2/2p1pn2/1p2P1B1/2pP4/2N3P1/PP3P1P/R2QKB1R b KQkq - 0 11")
stockfish.get_top_moves(5)

In [ ]:
import json
from analyser import *
def load_config(path: str) -> dict:
    with open(path) as file:
        raw_json_string = file.read()
        config = json.loads(raw_json_string)
        return config

def write_res_to_file(path: str, res: Dict):
    with open(path, "w") as file:
        file.write(res.__str__())


fen = "2r2b1k/1p1n4/1P1P4/p2Bp2q/1BN1Pp1n/R4PrP/5R1K/5Q2 w - - 0 38"
path = "output"

states = [(-float("inf"), -20), (-20, -10), (-10, -5), (-5, -3), (-3, -1), (-1, -0.5), (-0.5, 0.5), (0.5, 1), (1, 3), (3, 5), (5, 10), (10, 20), (20, float("inf"))]
states_including_end_points = [-1] + states + [1]
num_variations = 3
depth = 3
stockfish_depth = 20
config = load_config("stockfish_confg.json")

model = analyser(states, num_variations, depth, stockfish_depth, config)
adj, stoch, dist, expectation = model.eval_position(fen)
df = pd.DataFrame(stoch, columns=states_including_end_points, index=states_including_end_points)
print(expectation)

write_res_to_file(f"{path}/test/adj_", adj)
write_res_to_file(f"{path}/test/stoch_", df)
write_res_to_file(f"{path}/test/expectation_", expectation)
write_res_to_file(f"{path}/test/dist_.", dist)



In [ ]:
df

In [ ]:
adj

In [ ]:
mat = 0.95 * stoch + (1 - 0.95) * 1/15
mat

In [ ]:
mat ** 5

In [ ]:
3.175672e-01 * -4

In [ ]:
expectation

In [ ]:
temp = mat ** 5
temp

In [ ]:
temp[7]

In [ ]:
expectation = 0
for state, prob in zip(states[1:-1], temp[7][2:-2]):
    mid = (state[0] + state[1])/2
    print(state, prob)
    expectation += mid * prob

In [ ]:
expectation